In [3]:
!pip install moviepy gradio --quiet

In [ ]:
import os, cv2, numpy as np, math, random
from PIL import Image, ImageOps, ImageDraw
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import gradio as gr

# === Haar cascade for fallback detection ===
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

validated_points = {}  # {filename: [(x_left,y_left), (x_right,y_right)]}
current_file = None
clicked_points = []

# --- 1. Auto detect eyes (OpenCV only) ---
def detect_face_and_eyes(image_rgb):
    gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(100,100))
    if len(faces)==0: return None
    fx, fy, fw, fh = max(faces, key=lambda f: f[2]*f[3])
    roi_gray = gray[fy:fy+fh, fx:fx+fw]
    roi_color = image_rgb[fy:fy+fh, fx:fx+fw]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    if len(eyes)>=2:
        eyes = sorted(eyes, key=lambda e: e[0])
        ex1,ey1,ew1,eh1 = eyes[0]; ex2,ey2,ew2,eh2 = eyes[1]
        left_eye = (fx+ex1+ew1//2, fy+ey1+eh1//2)
        right_eye = (fx+ex2+ew2//2, fy+ey2+eh2//2)
        return {"left_eye": left_eye, "right_eye": right_eye}
    else:
        # fallback: estimate from face box
        cx = fx+fw/2; eye_y = fy+fh*0.37; sep = fw*0.25
        return {"left_eye": (int(cx-sep), int(eye_y)), "right_eye": (int(cx+sep), int(eye_y))}

# --- 2. Preview auto detection (fixed for Gradio) ---
def preview_auto(files):
    if not files or len(files) == 0:
        return None, "⚠️ No file uploaded"

    # Handle both dict and file-like objects
    file = files[0]
    path = file["name"] if isinstance(file, dict) else file.name

    try:
        pil_img = Image.open(path).convert("RGB")
        return pil_img, f"✅ Loaded {os.path.basename(path)}"
    except Exception as e:
        return None, f"❌ Error loading image: {e}"


# --- 3. Handle clicks ---
def on_click(evt: gr.SelectData):
    global clicked_points, current_file
    clicked_points.append(evt.index)
    if len(clicked_points) > 2:
        clicked_points = clicked_points[-2:]
    return str(clicked_points)

# --- 4. Validate ---
def validate_points():
    global current_file, clicked_points, validated_points
    if current_file and len(clicked_points)==2:
        validated_points[current_file] = clicked_points.copy()
        return f"✅ {os.path.basename(current_file)} validated {clicked_points}"
    return "⚠️ Please select 2 points (eyes)."

# --- 5. Align ---
def align_eyes_to_fixed_position(image_rgb, eyes, target_left, target_right, output_size=(1024,768)):
    left, right = np.array(eyes[0]), np.array(eyes[1])
    center = (left+right)/2.0
    t_center = (np.array(target_left)+np.array(target_right))/2.0
    angle = math.atan2((right-left)[1], (right-left)[0])
    t_angle = math.atan2((target_right-target_left)[1], (target_right-target_left)[0])
    rot = t_angle-angle
    h,w=image_rgb.shape[:2]
    M=cv2.getRotationMatrix2D(tuple(center), math.degrees(rot), 1.0)
    rotated=cv2.warpAffine(image_rgb,M,(w,h),flags=cv2.INTER_LANCZOS4,borderValue=(255,255,255))
    new_c=np.dot(M,[center[0],center[1],1])
    trans=t_center-new_c
    M2=np.float32([[1,0,trans[0]],[0,1,trans[1]]])
    return cv2.warpAffine(rotated,M2,output_size,flags=cv2.INTER_LANCZOS4,borderValue=(255,255,255))

# --- 6. Slideshow ---
def create_slideshow(files, music=None, dur=4.0, fade=2.0):
    os.makedirs("outputs", exist_ok=True)
    clips=[]; out_size=(1024,768)
    cx=out_size[0]//2; eye_y=out_size[1]//3; sep=140
    FIXED_L=(cx-sep//2,eye_y); FIXED_R=(cx+sep//2,eye_y)
    for f in files:
        if f.name not in validated_points: continue
        pil_img=Image.open(f.name).convert("RGB")
        white_bg=Image.new("RGB", pil_img.size,(255,255,255)); white_bg.paste(pil_img,(0,0))
        bordered=ImageOps.expand(white_bg,border=30,fill="white")
        aligned=align_eyes_to_fixed_position(np.array(bordered), validated_points[f.name], FIXED_L,FIXED_R,out_size)
        rotation=random.uniform(-4,4)
        clip=ImageClip(aligned).set_duration(dur).rotate(rotation)
        clips.append(clip)
    if not clips: return "❌ No validated images", None, None
    final=concatenate_videoclips(clips,method="compose",padding=-fade).crossfadein(fade)
    if music:
        try:
            audio=AudioFileClip(music.name).set_duration(final.duration)
            final=final.set_audio(audio)
        except: pass
    out="outputs/final_slideshow.mp4"
    final.write_videofile(out,fps=24,codec="libx264",audio_codec="aac",
                          ffmpeg_params=["-pix_fmt","yuv420p"],verbose=False,logger=None)
    final.close()
    return f"✅ Video created with {len(clips)} validated images", out, out

# === Gradio UI ===
with gr.Blocks() as demo:
    gr.Markdown("## 👁️ Validate eyes before slideshow")
    with gr.Row():
        file_in=gr.File(file_types=["image"], file_count="multiple", label="📸 Photos")
        music_in=gr.File(file_types=["audio"], label="🎵 Music (optional)")
    preview_btn=gr.Button("🔍 Preview first photo")
    img=gr.Image(type="pil", interactive=True, label="Click 2 points (eyes)")
    log=gr.Textbox(label="Log")
    preview_btn.click(preview_auto, inputs=[file_in], outputs=[img, log])
    img.select(on_click, outputs=log)
    validate_btn=gr.Button("✅ Validate this photo")
    validate_btn.click(validate_points, outputs=log)
    dur=gr.Slider(2,8,4,0.5,label="Duration per image (s)")
    fade=gr.Slider(1,4,2,0.5,label="Fade duration (s)")
    gen_btn=gr.Button("🎬 Generate video")
    out_txt=gr.Textbox()
    out_vid=gr.Video()
    out_file=gr.File()
    gen_btn.click(create_slideshow, inputs=[file_in,music_in,dur,fade], outputs=[out_txt,out_vid,out_file])

demo.launch(share=True, debug=True)
